In [183]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
#import matplotlib.pyplot as plt

start = datetime.datetime(2016, 1, 11)
end = datetime.datetime(2016, 12, 1)

#gs = web.DataReader("078930.KS", "yahoo", "2016-01-01", "2016-03-06")
#gs = web.DataReader("073490.KQ", "yahoo", start, end) #이노와이어리스
gs = web.DataReader("039130.KS", "yahoo", start, end) #하나투어

#print(gs.head(10))
gs.info()

new_gs = gs[gs['Volume'] != 0] #휴일제거


print(new_gs.head(10) )

short_window = 5
long_window = 10
signals = pd.DataFrame(index=new_gs.index)
signals['ma5'] = new_gs['Adj Close'].rolling(window=short_window).mean() #5일 평균종가
signals['ma20'] = new_gs['Adj Close'].rolling(window=long_window).mean() #20일 평균종가
signals['adj_percent'] = new_gs['Adj Close'].pct_change()
signals['volume_percent'] = new_gs['Volume'].pct_change()

#new_gs.insert(len(new_gs.columns), "MA5", ma5) #컬럼추가
#new_gs.insert(len(new_gs.columns), "MA20", ma20) #컬럼추가

#신호
signals['signal'] = 0.0

signals['signal'][short_window:] = \
    np.where( ( signals['ma5'][short_window:]>signals['ma20'][short_window:] ) , 1.0, 0.0)
signals['positions'] = signals['signal'].diff() #상 하향 cross를 구분한다. 

signals['buy'] = np.where( (signals['positions'] == 1.0) & \
                                ( signals['adj_percent'] >= 0.01 ), 1.0, 0.0 ) # buy 신호.

#0 NULL, 1 holding
status = 0

for index, row in signals.iterrows() :                      
    if status == 0 and row['buy'] == 1.0 :
        row['positions'] = 1.0
        status = 1
    elif status == 1 and row['positions'] == -1.0 :
        row['positions'] = -1.0
        status = 0
    else :
        row['positions'] = 0.0                
    
'''
signals['signal'][short_window:] = \
    np.where( ( signals['percent'][short_window:] < -0.02 ) | \
             ( signals['ma5'][short_window:] < signals['ma20'][short_window:] ) , 0.0, 1.0)
'''


#signals['signal_minus'][short_window:] = np.where( signals['ma5'][short_window:] - signals['ma20'][short_window:] < 3000, 1.0, 0.0 )

#print(signals.head(30))



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222 entries, 2016-01-11 to 2016-12-02
Data columns (total 6 columns):
Open         222 non-null float64
High         222 non-null float64
Low          222 non-null float64
Close        222 non-null float64
Adj Close    222 non-null float64
Volume       222 non-null int64
dtypes: float64(5), int64(1)
memory usage: 12.1 KB
                Open      High      Low    Close     Adj Close  Volume
Date                                                                  
2016-01-11  106000.0  106000.0  98800.0  99100.0  96303.203125  306106
2016-01-12   99100.0  100500.0  94700.0  96700.0  93970.937500  226571
2016-01-13   96300.0   97900.0  96300.0  96900.0  94165.296875  116738
2016-01-14   95000.0   96700.0  92600.0  95400.0  92707.617188  188718
2016-01-15   95800.0   96800.0  92100.0  93300.0  90666.890625  114347
2016-01-18   92200.0   92700.0  87700.0  88800.0  86293.890625  260671
2016-01-19   88000.0   88700.0  85800.0  88100.0  85613.

"\nsignals['signal'][short_window:] =     np.where( ( signals['percent'][short_window:] < -0.02 ) |              ( signals['ma5'][short_window:] < signals['ma20'][short_window:] ) , 0.0, 1.0)\n"

In [184]:
#https://plot.ly/pandas/line-charts/
import plotly.plotly as py
import plotly.graph_objs as go

buy = pd.DataFrame(index = signals.loc[signals.positions == 1.0 ].index)
buy['y'] = signals.ma5[signals.positions == 1.0 ]

sell = pd.DataFrame(index = signals.loc[signals.positions == -1.0 ].index)
sell['y'] = signals.ma5[signals.positions == -1.0 ]

#minus = pd.DataFrame(index = signals.loc[signals.signal_minus == 1.0 ].index)
#minus['y'] = signals.ma5[signals.signal_minus == 1.0 ]

data = [
    go.Scatter(        
        x=new_gs.index,
        y=new_gs['Adj Close'],
        mode='lines',
        name='Close'        
    ),
    go.Scatter(        
        x=signals.index,
        y=signals['ma5'],
        mode='lines',
        name='ma5'        
    ),
    go.Scatter(        
        x=signals.index,
        y=signals['ma20'],
        mode='lines',
        name='ma20'        
    ),
    go.Scatter(
        x=buy.index,
        y=buy['y'],
        marker={'color': 'red', 'symbol': 'plus', 'size': "10"},
        mode='markers',
        name='buy'  
    ),
    go.Scatter(
        x=sell.index,
        y=sell['y'],
        marker={'color': 'blue', 'symbol': 'minus', 'size': "10"},
        mode='markers',
        name='sell'  
    )
]

py.iplot(data, filename='stock')


In [189]:
# Set the initial capital
initial_capital= float(1000000.0)

# Create a DataFrame `positions`
positions = pd.DataFrame(index=signals.index).fillna(0.0)

# Buy a 100 shares
positions['AAPL'] = 10*signals['positions']   
#print("positions =>", positions )

# Initialize the portfolio with value owned   
portfolio = positions.multiply(new_gs['Adj Close'], axis=0)
portfolio['Adj Close'] = new_gs['Adj Close']

#print( portfolio.head(20) )

# Store the difference in shares owned 
pos_diff = positions.diff()

#print("pos_diff => ", pos_diff)

# Add `holdings` to portfolio
portfolio['holdings'] = ( positions.multiply(new_gs['Adj Close'], axis=0) ).sum( axis=1 )

#print( portfolio.head(30) )

# Add `cash` to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(new_gs['Adj Close'], axis=0)).sum(axis=1).cumsum()   

# Add `total` to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add `returns` to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

# Print the first lines of `portfolio`
print(portfolio.head(50))


                    AAPL      Adj Close      holdings          cash  \
Date                                                                  
2016-01-11       0.00000   96303.203125       0.00000  1.000000e+06   
2016-01-12       0.00000   93970.937500       0.00000  1.000000e+06   
2016-01-13       0.00000   94165.296875       0.00000  1.000000e+06   
2016-01-14       0.00000   92707.617188       0.00000  1.000000e+06   
2016-01-15       0.00000   90666.890625       0.00000  1.000000e+06   
2016-01-18       0.00000   86293.890625       0.00000  1.000000e+06   
2016-01-19       0.00000   85613.640625       0.00000  1.000000e+06   
2016-01-20       0.00000   83572.914063       0.00000  1.000000e+06   
2016-01-21       0.00000   84544.687500       0.00000  1.000000e+06   
2016-01-22       0.00000   89695.117188       0.00000  1.000000e+06   
2016-01-25       0.00000   88723.335938       0.00000  1.000000e+06   
2016-01-26       0.00000   91347.132813       0.00000  1.000000e+06   
2016-0

In [190]:
total = [
    go.Scatter(        
        x=portfolio.index,
        y=portfolio['total'],
        mode='lines',
        name='Total'        
    ),
    go.Scatter(        
        x=portfolio.index,
        y=portfolio['cash'],
        mode='lines',
        name='Cash'        
    ),
    go.Scatter(        
        x=portfolio.index,
        y=portfolio['holdings'],
        mode='lines',
        name='holdings'        
    )
    
]
py.iplot( total, filename='total')

크로스는 참조만 할 뿐 투자와는 전혀 연관이 없다.